<a href="https://colab.research.google.com/github/waterta0115/LDA_graduattion_thesis/blob/main/LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 準備

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import japanize_matplotlib
import arviz as az
import matplotlib.dates as mdates
from itertools import combinations
import igraph as ig
import warnings
warnings.filterwarnings('ignore')
az.style.use("arviz-doc")

In [ ]:
from __future__ import absolute_import, division, unicode_literals  # noqa
import logging
import sys
import lda._lda
import lda.utils
from scipy.special import digamma, polygamma

# ログ出力の設定(1)
logger = logging.getLogger('lda')

class LDA:
    def __init__(self, n_topics, n_iter=2000, update_alpha=False,update_eta=False,alpha=0.1, eta=0.01, random_state=None,
                 refresh=10):
        self.n_topics = n_topics
        self.n_iter = n_iter
        self.update_alpha = update_alpha
        self.alpha = alpha
        self.update_eta = update_eta
        self.eta = eta

        # if random_state is None, check_random_state(None) does nothing
        # other than return the current numpy RandomState
        self.random_state = random_state
        self.refresh = refresh

        alpha_is_valid = alpha > 0 if isinstance(alpha, (int, float)) else np.all(alpha > 0)
        if not alpha_is_valid or eta <= 0:
             raise ValueError("alpha and eta must be greater than zero")

        # random numbers that are reused
        rng = lda.utils.check_random_state(random_state)
        self._rands = rng.rand(1024**2 // 8)  # 1MiB of random variates

        # configure console logging if not already configured
        # ログ出力の設定(2)
        if len(logger.handlers) == 1 and isinstance(logger.handlers[0], logging.NullHandler):
            logging.basicConfig(level=logging.INFO)

    def fit(self, X, y=None):
        self._fit(X)
        return self

    def fit_transform(self, X, y=None):
        if isinstance(X, np.ndarray):
            # in case user passes a (non-sparse) array of shape (n_features,)
            # turn it into an array of shape (1, n_features)
            X = np.atleast_2d(X)
        self._fit(X)
        return self.doc_topic_

    def transform(self, X, max_iter=20, tol=1e-16):
        if isinstance(X, np.ndarray):
            # in case user passes a (non-sparse) array of shape (n_features,)
            # turn it into an array of shape (1, n_features)
            X = np.atleast_2d(X)
        doc_topic = np.empty((X.shape[0], self.n_topics))
            # 結果格納用の配列を用意
        WS, DS = lda.utils.matrix_to_lists(X)
            # 行列→リスト　WS：単語インデックス　DS：その単語がどの文書に属しているか　に変換
        """
        Doc1: 単語1が2回, 単語3が1回
        Doc2: 単語2が3回
        """
        # この場合
        # WS = [1, 1, 3, 2, 2, 2]
        # DS = [0, 0, 0, 1, 1, 1]
        # このようにWSとDSは同じ長さとなる

        # TODO: this loop is parallelizable
        for d in np.unique(DS):
            doc_topic[d] = self._transform_single(WS[DS == d], max_iter, tol)
        return doc_topic

    def _transform_single(self, doc, max_iter, tol):
        # 内部的に doc = WS[DS == d] のような変換が行われている(あるdに属している単語集合)
        # ここでdocは「文書中の単語たち」を表す 単語インデックスの配列
        PZS = np.zeros((len(doc), self.n_topics))
        for iteration in range(max_iter + 1):  # +1 is for initialization
            PZS_new = self.components_[:, doc].T
                # components_はトピック-単語分布行列(phi)
                # その中からdoc内に含まれている単語だけを抽出して転置
                # PZS_newは各単語が「どのトピックにどれくらい関係しているか」の確率表
            PZS_new *= (PZS.sum(axis=0) - PZS + self.alpha)
                # PZS.sum(axis=0)：各文書はどれくらいの数のトピックを持っているか
                    # ここではブロードキャスト機能が自動的に働くため、PZSの次元は(1,n_topics)となる
                # PZS.sum(axis=0)-PZS：shape==(n_words_in_doc,n_topics)
                # self.alphaはスカラー又は(n_topics,)であるが、ブロードキャストにより全体の次元は(n_words_in_doc,n_topics)
            PZS_new /= PZS_new.sum(axis=1)[:, np.newaxis]  # vector to single column matrix
                # np.array([a,b,c])[:,np.newaxis]とすることで1次元配列を2次元配列にすることが可能
                    # (3,)→(3,1)；これらは全くの別物
                # ここでは確率にしている（正規化）
            delta_naive = np.abs(PZS_new - PZS).sum()
                # 前回の値との変化量
            logger.debug('transform iter {}, delta {}'.format(iteration, delta_naive))
            PZS = PZS_new
            if delta_naive < tol:
                break
                # 収束判定
        theta_doc = PZS.sum(axis=0) / PZS.sum()
            # 文書内のすべての単語にわたるトピック確率を合計し、全体で正規化して文書レベルのトピック分布を作る
        assert len(theta_doc) == self.n_topics
        assert theta_doc.shape == (self.n_topics,)
        return theta_doc

    def _fit(self, X):
        """inference by """
        random_state = lda.utils.check_random_state(self.random_state)
        rands = self._rands.copy()
            # randsは事前に生成された乱数列のコピー
            # gibbs samplingで使う乱数をここから供給する
        self._initialize(X)
            # 各単語にランダムにトピックを割り当てる
            # 各カウント行列を作成：nzw_,ndz_,nz_
                # nzw_：topic-wordの共起数
                # ndz_：doc-topicの共起数
                # nz_：各トピックに属する単語数の合計
        for it in range(self.n_iter):
            # FIXME: using numpy.roll with a random shift might be faster
            random_state.shuffle(rands)
            if it % self.refresh == 0:
                ll = self.loglikelihood()
                logger.info("<{}> log likelihood: {:.0f}".format(it, ll))
                # keep track of loglikelihoods for monitoring convergence
                self.loglikelihoods_.append(ll)
            self._sample_topics(rands)
            if it == 1:
                print("ndz 初回サンプル:",self.ndz_[:5])
            if it == 100:
                print("ndz 100回目:", self.ndz_[:5])
            if self.update_alpha:     # ユーザが制御できるようにオプション化
                self._update_alpha()
                if it % 10 == 0:
                    logger.info(f"iteration{it}: alpha = {self.alpha}")
            if self.update_eta:     # ユーザが制御できるようにオプション化
                self._update_eta()
                if it % 10 == 0:
                    logger.info(f"iteration{it}: alpha = {self.eta}")
            # ここがcollapsed gibbs samplingの実体。lda-projectの内部で構築されている
        ll = self.loglikelihood()
            # 最終的な対数尤度の計算・出力
        logger.info("<{}> log likelihood: {:.0f}".format(self.n_iter - 1, ll))
        # note: numpy /= is integer division
        self.components_ = (self.nzw_ + self.eta).astype(float)
            # components_とは各トピックごとの単語分布（phi）：(n_topics,n_words)
            # etaを加えることでスムージングしている
        self.components_ /= np.sum(self.components_, axis=1)[:, np.newaxis]
            # 確率にしている（正規化）
        self.topic_word_ = self.components_
            # scikit-learn互換用のために同じ内容を別名で保存
        self.doc_topic_ = (self.ndz_ + self.alpha).astype(float)
            # doc_topic_とは各文書ごとのトピック分布(theta)：(n_docs,n_topics)
        self.doc_topic_ /= np.sum(self.doc_topic_, axis=1)[:, np.newaxis]

        # delete attributes no longer needed after fitting to save memory and reduce clutter
        del self.WS
        del self.DS
        del self.ZS
        return self
            # components_とdoc_topic_とloglikelihoods_を返す

    def _initialize(self, X):
        D, W = X.shape
        N = int(X.sum())
        n_topics = self.n_topics
        n_iter = self.n_iter
        logger.info("n_documents: {}".format(D))
        logger.info("vocab_size: {}".format(W))
        logger.info("n_words: {}".format(N))
        logger.info("n_topics: {}".format(n_topics))
        logger.info("n_iter: {}".format(n_iter))

        self.nzw_ = nzw_ = np.zeros((n_topics, W), dtype=np.intc, order="F")
        self.ndz_ = ndz_ = np.zeros((D, n_topics), dtype=np.intc, order="C")
        self.nz_ = nz_ = np.zeros(n_topics, dtype=np.intc)

        self.WS, self.DS = WS, DS = lda.utils.matrix_to_lists(X)
        self.ZS = ZS = np.empty_like(self.WS, dtype=np.intc)
        np.testing.assert_equal(N, len(WS))
        for i in range(N):
            w,d = WS[i], DS[i]
            z_new = i % n_topics
            ZS[i] = z_new
            ndz_[d, z_new] += 1
            nzw_[z_new, w] += 1
            nz_[z_new] += 1
        self.loglikelihoods_ = []

    def loglikelihood(self):# ここではalphaやetaはスカラーであることが想定されている
        nzw, ndz, nz = self.nzw_, self.ndz_, self.nz_
        alpha = self.alpha
        eta = self.eta
        nd = np.sum(ndz, axis=1).astype(np.intc)
        if isinstance(alpha, np.ndarray) and alpha.ndim > 0:
            alpha_scalar = alpha[0]
        else:
            alpha_scalar = alpha
        if isinstance(eta, np.ndarray) and eta.ndim > 0:
            eta_scalar = eta[0]
        else:
            eta_scalar = eta
        return lda._lda._loglikelihood(nzw, ndz, nz, nd, alpha_scalar, eta_scalar)

    def _sample_topics(self, rands):# ここではalphaやetaはK次元であることが想定されている
        n_topics, vocab_size = self.nzw_.shape
        if isinstance(self.alpha, (int, float)):
            alpha_array = np.full(n_topics, self.alpha)
        else:
            alpha_array = self.alpha
        if isinstance(self.eta, (int, float)):
            eta_array = np.full(n_topics, self.eta)
        else:
            eta_array = self.eta
        alpha = alpha_array.astype(np.float64)
        eta = eta_array.astype(np.float64)
        lda._lda._sample_topics(self.WS, self.DS, self.ZS, self.nzw_, self.ndz_, self.nz_,
                                alpha, eta, rands)

    import numpy as np
    from scipy.special import digamma, polygamma
    def _update_alpha(self):
        if not hasattr(self, "ndz_"):
            raise ValueError("ndz_ が存在しません")

        if np.all(self.ndz_ == self.ndz_[0]):
            print("警告: ndz_ が全ドキュメント同じ値になっています")

        dt = self.ndz_              # shape (D, K)
        D, K = dt.shape

        # ensure alpha is scalar
        if isinstance(self.alpha, np.ndarray):
            alpha_scalar = float(self.alpha[0])
        else:
            alpha_scalar = float(self.alpha) # float型の場合はそのままfloatに変換

        # s = digamma(dt + alpha_scalar).sum(axis=0)
        # g = s - D * digamma(alpha_scalar)
        # h = -D * polygamma(1, alpha_scalar)
        # alpha_new = alpha_scalar - g / h
        for i in range(100):
            N_d = np.sum(dt,axis=1)
            term1_num = np.sum(digamma(dt+alpha_scalar))
            term2_num = D*K*digamma(alpha_scalar)
            numerator = term1_num - term2_num
            term1_den = K*np.sum(digamma(N_d+alpha_scalar*K))
            term2_den = D*K*digamma(alpha_scalar*K)
            denominator = term1_den - term2_den
            alpha_new = alpha_scalar*(numerator/denominator)
            diff = np.abs(alpha_new-alpha_scalar)
            print(f"Iter{i+1}:alpha={alpha_new:.6f}(diff={diff:.6e})")
            if diff<1e-05:
                print("収束しました")
                break
            alpha_scalar = alpha_new
        # store symmetric vector
        self.alpha = np.ones(K)*alpha_new

        # logging
        logger.info(f"alpha updated (symmetric): {alpha_new}")

        if not hasattr(self, "alpha_history"):
            self.alpha_history = []
        self.alpha_history.append(self.alpha.copy())
    def _update_eta(self):
        if not hasattr(self, "nzw_"):
            raise ValueError("nzw_ が存在しません")

        nzw = self.nzw_              # shape (K, V)
        nz = self.nz_               # shape (K,)
        K, V = nzw.shape

        # ensure eta is scalar
        if isinstance(self.eta, np.ndarray):
            eta_scalar = float(self.eta[0])
        else:
            eta_scalar = float(self.eta)

        # fixed-point iteration (symmetric eta)
        for i in range(100):
            term1_num = np.sum(digamma(nzw + eta_scalar))     # Σ_k Σ_w ψ(n_kw + η)
            term2_num = K * V * digamma(eta_scalar)           # K*V ψ(η)
            numerator = term1_num - term2_num

            term1_den = V * np.sum(digamma(nz + V * eta_scalar))  # V Σ_k ψ(n_k· + Vη)
            term2_den = K * V * digamma(V * eta_scalar)           # K*V ψ(Vη)
            denominator = term1_den - term2_den

            eta_new = eta_scalar * (numerator / denominator)
            diff = abs(eta_new - eta_scalar)

            print(f"[eta] Iter{i+1}: eta={eta_new:.6f} (diff={diff:.6e})")

            if diff < 1e-5:
                print("eta 収束しました")
                break

            eta_scalar = eta_new

        # store symmetric eta vector
        self.eta = eta_new

        # logging
        logger.info(f"eta updated (symmetric): {eta_new}")

        # store history
        if not hasattr(self, "eta_history"):
            self.eta_history = []
        self.eta_history.append(eta_new)

In [1]:
def prepare(df):
    from gensim import corpora
    customer_texts = df.groupby("顧客ID")["商品カテゴリ"].apply(list).tolist()
    dictionary = corpora.Dictionary(customer_texts)
    corpus = [dictionary.doc2bow(text) for text in customer_texts]
    return customer_texts,dictionary,corpus

In [ ]:
def make_bow(posdata):
    from sklearn.feature_extraction.text import CountVectorizer

# 1. 仮統合顧客番号ごとに、品目名を空白区切りで連結した「文書」として扱う
    customer_docs = (
        posdata
        .groupby("顧客ID")["商品カテゴリ"]
        .apply(lambda items: ' '.join(map(str, items)))
    )

# 2. CountVectorizerでBOW作成
    vectorizer = CountVectorizer(token_pattern='[^ ]+')
    bow_matrix = vectorizer.fit_transform(customer_docs)
    vocab = vectorizer.get_feature_names_out()
# n_samples, n_features
    print("BOW shape:", bow_matrix.shape)
    return bow_matrix, vocab

# 実行

In [ ]:
texts,dictionary,corpus = make_bow(posdata)

In [ ]:
from tqdm import tqdm
from gensim.models.coherencemodel import CoherenceModel

start = 2
limit = 50
step = 2
perplexity_vals = []
coherence_vals = []

BOW,vocab = make_bow(posdata)

for n_topic in tqdm(range(start, limit, step), desc="Calculating perplexity"):
    lda = LDA(n_topic,alpha=0.1,eta=0.01,update_alpha=True,update_eta=True, random_state = 0, n_iter = 1500)
    lda.fit(BOW)

    loglik = lda.loglikelihood()    # ← ここが重要！
    total_words = np.sum(BOW)
    perplexity = np.exp(-loglik / total_words)
    perplexity_vals.append(perplexity)
    # --- Coherence (gensim の CoherenceModel を利用) ---
    topic_word = lda.topic_word_   # shape (n_topics, vocab)
    topics = []
    n_top_words = 5
    for t in range(n_topic):
        # topn_uni = topic_word_uni[t].argsort()[:-n_top_words - 1:-1]
        # topics_uni.append([vocab_uni[w_id] for w_id in topn_uni])
        word_ids = topic_word[t].argsort()[-n_top_words:][::-1]
        word_ids = [int(i) for i in word_ids]  # numpy型→python int
        words = [dictionary[i] for i in word_ids]
        topics.append(words)
    cm = CoherenceModel(topics=topics, texts=texts, dictionary=dictionary, coherence='c_npmi')
    coherence_vals.append(cm.get_coherence())

x = range(start, limit, step)
fig, ax1 = plt.subplots(figsize=(12,5))
c1 = 'darkturquoise'
ax1.plot(x, coherence_vals, 'o-', color=c1)
ax1.set_xlabel('Num Topics')
ax1.set_ylabel('Coherence', color=c1); ax1.tick_params('y', colors=c1)
c2 = 'slategray'
ax2 = ax1.twinx()
ax2.plot(x, perplexity_vals, 'o-', color=c2)
ax2.set_ylabel('Perplexity', color=c2); ax2.tick_params('y', colors=c2)

ax1.set_xticks(x)
plt.show()

In [ ]:
lda = LDA(n_topics = 6 , random_state = 0, n_iter = 1500)
lda.fit(BOW)
topic_word = lda.components_  # model.components_ also works
n_top_words = 6
plt.plot(lda.loglikelihoods_[5:])
plt.show()

In [ ]:
import pyLDAvis

topic_term_dists = lda.topic_word_   # shape: (n_topics, n_terms)
doc_topic_dists = lda.doc_topic_     # shape: (n_docs, n_topics)
term_frequency = np.asarray(BOW.sum(axis=0)).flatten()  # 各単語の出現回数

# --- pyLDAvis形式に変換して可視化 ---
vis_data = pyLDAvis.prepare(
    topic_term_dists=topic_term_dists,
    doc_topic_dists=doc_topic_dists,
    doc_lengths=BOW.sum(axis=1).A1,
    vocab=vocab,
    term_frequency=term_frequency
)
pyLDAvis.display(vis_data)